In [1]:
import pandas as pd
import numpy as np
import math
import time

In [2]:
S_t = 50
Save_t = 50
K = 40
r = 0.3
q = 0.1
Sigma = 0.4
t = 0.1
T = 0.5
n_crr = 100
M = 50

In [3]:
#預先算好這些共用參數
delta_t = (T - t)/n_crr
u = np.exp(Sigma * np.sqrt(delta_t))
d = 1/u #算p比較方便，後面不會用到
p = (np.exp((r - q) * delta_t) - d) / (u - d)

# 4種方法

In [4]:
methods = ["Buildin", "Sequential", "Binary", "Linear"]

In [5]:
def generate_panel(n):
    #d = [[0 for i in range(100)] for j in range(100)]
    d = np.zeros((n, n))
    data = np.array(d, dtype=object)
    for i in range(n):
        for j in range(n):
            data[i][j] = list()
    return data
#==================================================================================================================#
def count_data_manage(n, df):
    if n in df.keys():
        df[n] += 1
    else:
        df[n] = 1
#####################################################################################################################
def Sequential_search_count(x, list_x, list_y):
    start_t = time.time()
    count = -1
    for i in list_x:
        if i >= x:
            count += 1
        else:
            break
    if count == -1:
        c = list_y[0]
    elif count == len(list_y)-1:
        c = list_y[-1]
    else:
        wu = (list_x[count] - x) / (list_x[count] - list_x[count+1])
        c = wu * list_y[count+1] + (1-wu) * list_y[count]
    end_t = time.time()
    out_time = end_t - start_t
    return c, count + 2, out_time #count是總共花幾次找到答案
#####################################################################################################################
def midpoint(a, b):
    return min(a, b) + (max(a, b) - min(a, b)) // 2
#------------------------------------------------------------------------------------------------------------------#    
def process_plus_minus(x, n, l, l_2, l_3): #n是index
    result = l[n] - x
    if result == 0:
        l_2.append(0)
    elif result > 0:
        l_2.append(1)
    else:
        l_2.append(-1)
    l_3.append(n)
#------------------------------------------------------------------------------------------------------------------#
def find_diff(a):
    for i in range(-1, -len(a)-1, -1):
        if a[i] != a[-1]:
            break
    return i
#------------------------------------------------------------------------------------------------------------------#
def Binary_search_count(x, list_x, list_y):
    start_t = time.time()
    
    seq_l = []
    plus_minus_l = []
    
    process_plus_minus(x, 0, list_x, plus_minus_l, seq_l)
    if plus_minus_l[-1] in [0, -1]:
        c = list_y[0]
        end_t = time.time()

    else:
        process_plus_minus(x, len(list_x)-1, list_x, plus_minus_l, seq_l)
        if plus_minus_l[-1] in [0, 1]:
            c = list_y[-1]
            end_t = time.time()

        else:
            for i in range(math.ceil(math.log(len(list_x) - 1, 2))):
                if plus_minus_l[-1] == 0:
                    c = list_y[seq_l[-1]]
                    end_t = time.time()
                    break
                
                elif (1) not in plus_minus_l[1:]:
                    tem_num = midpoint(seq_l[-1], seq_l[0]) #如果一路找下來都是負
                
                elif (-1) not in plus_minus_l[1:]:
                    tem_num = midpoint(seq_l[-1], seq_l[1]) #如果一路找下來都是正
                
                else:
                    if i == 0: #第一輪直接從最上最下中間抽
                        tem_num = midpoint(seq_l[-1], seq_l[-2]) #如果確定在前兩個的中間(前兩個一正一負)
                    
                    else:
                        if plus_minus_l[-1] * plus_minus_l[-2] == (-1): #如果倒數兩個異號
                            tem_num = midpoint(seq_l[-1], seq_l[-2]) #如果確定在前兩個的中間(前兩個一正一負)
                        
                        if plus_minus_l[-1] * plus_minus_l[-2] == (1) : #如果最後至少兩個同號而前面有不同號的
                            nex = find_diff(plus_minus_l) #找到跟倒數第一個不同號的
                            if (seq_l[-1] == (seq_l[nex] - 1)) | (seq_l[-1] == (seq_l[nex] + 1)): #如果前面已經有只差1的，那代表已經找完了
                                wu = (list_x[min(seq_l[-1], seq_l[nex])] - x) / (list_x[min(seq_l[-1], seq_l[nex])] - list_x[max(seq_l[-1], seq_l[nex])])
                                c = wu * list_y[max(seq_l[-1], seq_l[nex])] + (1-wu) * list_y[min(seq_l[-1], seq_l[nex])]
                                end_t = time.time()
                                break
                                
                            else:
                                tem_num = midpoint(seq_l[-1], seq_l[nex]) #如果沒有只差1號的，就要再繼續找
                                #print("yes")
                        
                process_plus_minus(x, tem_num, list_x, plus_minus_l, seq_l)
            
            if i == math.ceil(math.log(len(list_x) - 1, 2)) - 1:
                if (seq_l[-1] == (seq_l[-2] - 1)) | (seq_l[-1] == (seq_l[-2] + 1)):
                    wu = (list_x[min(seq_l[-1], seq_l[-2])] - x) / (list_x[min(seq_l[-1], seq_l[-2])] - list_x[max(seq_l[-1], seq_l[-2])])
                    c = wu * list_y[max(seq_l[-1], seq_l[-2])] + (1-wu) * list_y[min(seq_l[-1], seq_l[-2])]
                    end_t = time.time()

                if (seq_l[-1] == (seq_l[-3] - 1)) | (seq_l[-1] == (seq_l[-3] + 1)):
                    wu = (list_x[min(seq_l[-1], seq_l[-3])] - x) / (list_x[min(seq_l[-1], seq_l[-3])] - list_x[max(seq_l[-1], seq_l[-3])])
                    c = wu * list_y[max(seq_l[-1], seq_l[-3])] + (1-wu) * list_y[min(seq_l[-1], seq_l[-3])]
                    end_t = time.time()
    
    #end_t = time.time()
    out_time = end_t - start_t
    return c, len(seq_l), out_time
    #return seq_l, plus_minus_l
#####################################################################################################################
def Linear_interpolation_count(x, list_x, list_y):
    start_t = time.time()
    
    if list_x[0] == list_x[-1]:
        c = list_y[0]
        count = 1
    
    else:
        if list_x[0] <= x:
            c = list_y[0]
            count = 1

        elif list_x[-1] >= x:
            c = list_y[-1]
            count = 2

        else:
            diff = (list_x[0] - list_x[-1]) / (len(list_x)-1)
            num = (x - list_x[-1]) / diff
            if num == int(num): #如果剛好落在某個點上
                c = list_y[len(list_x) - int(num)]
                count = 3

            else:
                d_n = (len(list_x)-1) - math.floor(num)
                u_n = (len(list_x)-1) - math.ceil(num)
                wu = (list_x[u_n] - x) / (list_x[u_n] - list_x[d_n])
                c = wu * list_y[d_n] + (1-wu) * list_y[u_n]
                count = 3
    
    end_t = time.time()
    out_time = end_t - start_t
    return c, count, out_time
    #return c, count, out_time, u_n, d_n
#####################################################################################################################
def Build_in_func_count(x, list_x, list_y):
    start_t = time.time()
    c = np.interp(x, list_x[::-1], list_y[::-1])
    end_t = time.time()
    out_time = end_t - start_t
    return c, 1, out_time
#####################################################################################################################
def All_methods(x, list_x, list_y, method = "Buildin"):
    if method == "Buildin":
        c, count, time = Build_in_func_count(x, list_x, list_y)
    if method == "Sequential":
        c, count, time = Sequential_search_count(x, list_x, list_y)
    if method == "Binary":
        c, count, time = Binary_search_count(x, list_x, list_y)
    if method == "Linear":
        c, count, time = Linear_interpolation_count(x, list_x, list_y)
    return c, count, time
#####################################################################################################################
def Binomial_Arithmetic_call_count(M = 50):
    #建立資料panel
    panel = generate_panel(n_crr + 1)

    #第一步：算出panel上每個node的Amax, Amin
    for col in range(n_crr + 1):
        for row in range(col + 1):
            Amax = (Save_t * t * n_crr + S_t * ((u**(col-row) * (u**(-row)-u-1) + 1) / (1-u))) / (t * n_crr + (col+1))
            Amin = (Save_t * t * n_crr + S_t * ((u**(-row) * (u-u**(col-row+1)+1) - u) / (1-u))) / (t * n_crr + (col+1))
            panel[row][col].append([Amax])
            panel[row][col].append([Amin])

    #第二步：跟據M將每個node的Amax到Amin中間補上M-1個等差A(i,j,k)
    for col in range(n_crr+1):
        for row in range(col+1):
            #如果Amax=Amin就不要做
            if panel[row][col][0][0] == panel[row][col][-1][0]:
                pass
            else:
                for k in range(1, M):
                    ins = ((M-k)/M) * panel[row][col][0][0] + (k/M) * panel[row][col][-1][0]
                    #ins = np.exp(((M-k)/M) * np.log(panel[row][col][0][0]) + (k/M) * np.log(panel[row][col][-1][0]))
                    panel[row][col].insert(k, [ins])

    #第三步：將最後一期算出payoff
    for row in range(n_crr + 1):
        for i in panel[row][-1]:
            call_price = max(i[0]-K,0)
            i.append(call_price)
               
    return panel

# 進行20次算時間，每次都跑四種方法

In [6]:
total_count_time = {k:[] for k in methods}
for times in range(20):
    
    panel = Binomial_Arithmetic_call_count(50)
    total_count_dict = {}
    for method in methods:

        #建立用來紀錄花幾次找到答案的字典，以及儲存所有時間的變數
        count_dict = {}
        count_time = 0

        #第四步：backward induction，對每個node中每個A(i,j,k)分別計算call price
        for col in range(n_crr - 1, -1, -1):
            for row in range(col+1):
                for i in range(len(panel[row][col])):

                    Au = (panel[row][col][i][0] * (t * n_crr + (col+1)) + S_t * u **(col-2*row+1)) / (t * n_crr + (col+1) + 1)
                    Ad = (panel[row][col][i][0] * (t * n_crr + (col+1)) + S_t * u **(col-2*row-1)) / (t * n_crr + (col+1) + 1)

                    #用歐式選擇權做代表

                    u_call_price, u_count, u_time = All_methods(Au, [j[0] for j in panel[row][col+1]], [j[1] for j in panel[row][col+1]], method)
                    d_call_price, d_count, d_time = All_methods(Ad, [k[0] for k in panel[row+1][col+1]], [k[1] for k in panel[row+1][col+1]], method)

                    call_price = np.exp(-r * delta_t) * (p * u_call_price + (1-p) * d_call_price)
                    panel[row][col][i].append(call_price)

                    #前面都照做，到最後才區分為最上面一條＆最下面一條以及其他，若非最上或最下，每個node會有M+1個點
                    #如果是最上或最下，因為不是最大就是最小，加入會扭曲結果，所以選擇不紀錄
                    if len(panel[row][col]) == 2:
                        #total_time += u_time
                        #total_time += d_time
                        continue

                    #如果不是最上或最下，除了最後一個column，每個node有M+1個點，每個點會貢獻兩筆資料
                    else:
                        count_data_manage(u_count, count_dict)
                        count_data_manage(d_count, count_dict)
                        count_time += u_time
                        count_time += d_time

        total_count_dict[method] = count_dict
        total_count_time[method].append(count_time)
    print(times, " Complete")

/Users/Kang/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:148: RuntimeWarning: divide by zero encountered in double_scalars
/Users/Kang/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:149: RuntimeWarning: invalid value encountered in double_scalars


0  Complete
1  Complete
2  Complete
3  Complete
4  Complete
5  Complete
6  Complete
7  Complete
8  Complete
9  Complete
10  Complete
11  Complete
12  Complete
13  Complete
14  Complete
15  Complete
16  Complete
17  Complete
18  Complete
19  Complete


In [7]:
total_count_time

{'Buildin': [4.900104761123657,
  4.791133165359497,
  4.593521356582642,
  4.954737424850464,
  4.599141597747803,
  4.712076425552368,
  4.799771547317505,
  4.809939861297607,
  4.750027656555176,
  4.671769857406616,
  4.765645742416382,
  4.776225566864014,
  4.666220188140869,
  4.662822723388672,
  4.743956565856934,
  4.6452672481536865,
  4.639033079147339,
  4.616406202316284,
  4.8438780307769775,
  4.671027898788452],
 'Sequential': [1.2534551620483398,
  1.2643654346466064,
  1.170027256011963,
  1.196265697479248,
  1.1194024085998535,
  1.2377302646636963,
  1.1235287189483643,
  1.238126516342163,
  1.163419485092163,
  1.1554784774780273,
  1.15794038772583,
  1.12762451171875,
  1.1258091926574707,
  1.122485637664795,
  1.113877296447754,
  1.116837501525879,
  1.132028579711914,
  1.12416672706604,
  1.1684505939483643,
  1.1539795398712158],
 'Binary': [8.007206439971924,
  8.782124996185303,
  8.991153001785278,
  8.287054538726807,
  8.24468445777893,
  8.4638345

# 平均運算次數以及平均計算總時間

In [8]:
df = pd.DataFrame(columns = methods, index = ["平均次數", "平均時間"])
for i in range(len(methods)):
    df.loc["平均次數", methods[i]] = sum(k*v for k, v in total_count_dict[methods[i]].items()) / sum(total_count_dict[methods[i]].values())
    df.loc["平均時間", methods[i]] = np.mean(total_count_time[methods[i]])

In [9]:
df

,Buildin,Sequential,Binary,Linear
平均次數,1,26.6631,7.70951,2.96439
平均時間,4.73064,1.16325,8.29909,1.07252
